#### Using the EPA Categories developed by G Helman which augment the implementation in the OECD Toolbox Version 4.5

<h1>Extract SMARTS</h1>

In [1]:
import pandas as pd
import numpy as np
import pymongo
import sys
import os
from datetime import datetime

from rdkit import Chem

import pymongo


In [2]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'

In [3]:
with open(raw_dir + 'epa_categories.xml','r') as f:
    xml=f.read()

In [4]:
xml=xml.replace('\n','')

In [5]:
#xml

In [6]:
import xml.etree.ElementTree as ET
e=ET.parse(raw_dir+'epa_categories.xml').getroot()

In [7]:
e

<Element '{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Scheme' at 0x7f6336c8fe50>

In [8]:
parent_map = {c:p for p in e.iter() for c in p}

In [9]:
len(parent_map)

4545

In [10]:
import operator as op
op_dict={
    'GreaterThan': op.gt,
    'GreaterThanOrEqualTo': op.ge,
    'LessThan': op.lt,
    'LessThanOrEqualTo': op.le
}
prop_dict={
    'log Kow':'logp',
    'Molecular Weight':'mol_weight',
    'Molecular weight':'mol_weight',
    'Water Solubility': 'ws'
}

In [11]:
def define_smart_match(smart):
    pattern=Chem.MolFromSmarts(smart)
    if not pattern:
        return None
    def smart_match(x):
        mol=x['mol']
        ret=True if mol.GetSubstructMatches(pattern) else False
        return ret
    return smart_match
def define_compare(prop,operand,value):
    def compare(x):
        ret = op_dict[operand](x[prop_dict[prop]],value)
        return ret
    return compare

In [12]:
class Query:
    
    def __init__(self,xml,qid=None):
        self.xml=xml
        self.id=qid
        self.logic=None
        self.subqueries=[]
        self.category=None
        
    def write_query(self,qtype,tree):
        self.type=qtype
        if qtype=='b:StructureQuery':
            qstring=self.xml.find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Queries}ComplexSearch').text
            qstring=re.sub('false','False',qstring)
            qstring=re.sub('true','True',qstring)
            qdict=ast.literal_eval(qstring)
            smart=qdict['queries'][0]['smart']
            self.smart=smart
            if '[Ch3,#1]' in self.smart:
                split=re.search(r'(.*)\[([^\(\)]*),([^\(\)].*)\]$',self.smart)
                split1=split.group(1)
                split2=split.group(1)+'['+split.group(2)+']'
                smart_match1=define_smart_match(split1)
                smart_match2=define_smart_match(split2)
                def smart_match(x):
                    return any([smart_match1(x),smart_match2(x)])
            else:
                smart_match=define_smart_match(smart)
            self.query=smart_match
        elif qtype=='b:ParameterQuery':
            self.operand=self.xml.find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Queries}Operand').text
            self.prop=self.xml.find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Queries}ParameterName').text
            self.value=float(self.xml.find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Queries}Value').text)
            compare=define_compare(self.prop,self.operand,self.value)
            self.query=compare
        elif qtype=='LogicalQuery':
            self.logic=self.xml.find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Logic').text
            elements=self.xml.find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Elements')
            node_ids=[elem.attrib['{http://schemas.microsoft.com/2003/10/Serialization/}Ref']\
                      for elem in elements.findall('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Query')\
                      if '{http://schemas.microsoft.com/2003/10/Serialization/}Ref' in elem.attrib]
            if self.logic=='Not':
                node_id=node_ids[0] #Should only be one
                sq=tree[node_id]
                self.subqueries=[sq]
                def func(x):
                    return not(sq.query(x))
                self.query=func
            elif self.logic=='And':
                sqs=[tree[node_id] for node_id in node_ids]
                self.subqueries=sqs
                def func(x):
                    return all([sq.query(x) for sq in self.subqueries])
                self.query=func
            else:
                sqs=[tree[node_id] for node_id in node_ids]
                self.subqueries=sqs
                for orquery in elements.findall('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Query'):
                    if '{http://www.w3.org/2001/XMLSchema-instance}type' in orquery.attrib:
                        extra_sq=Query(orquery)
                        extra_sq.write_query('b:StructureQuery',tree)
                        sqs.append(extra_sq)      
                def func(x):
                    return any([sq.query(x) for sq in self.subqueries])
                self.query=func
    
    def print_tree(self,x,tabs=0):
        qinfo=(self.id,self.type)
        if self.type=='b:StructureQuery':
            qinfo=qinfo+(self.smart,)
        elif self.type=='b:ParameterQuery':
            qinfo=qinfo+(self.prop,self.value,self.operand)
        elif self.type=='LogicalQuery':
            qinfo=qinfo+(self.logic,)
        try:
            qinfo=qinfo+(self.query(x),)
        except:
            qinfo=qinfo+('does not process',)
        print('\t'*tabs+str(qinfo))
        for sq in self.subqueries:
            sq.print_tree(x,tabs+1)

In [13]:
q = Query(xml)

In [15]:
all_tests={}
print_tests = {}
bad_smarts=set()
bad_cats=set()
import re
import ast
for elem in e.iter('{http://schemas.microsoft.com/2003/10/Serialization/Arrays}anyType'):
    category=elem.find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Caption').text
    queries=elem.find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Expression')\
        .find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Queries')\
        .findall('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Query')
    contents=[query.find('{http://schemas.datacontract.org/2004/07/LMC.Profiling.Engine}Content') for query in queries]
    #print(query)
    query_tree={}
    for query in contents:
        attributes=query.attrib
        if '{http://schemas.microsoft.com/2003/10/Serialization/}Id' not in attributes:
            continue
        query_id=attributes['{http://schemas.microsoft.com/2003/10/Serialization/}Id']
        query_type=attributes['{http://www.w3.org/2001/XMLSchema-instance}type']
        q=Query(query,query_id)
        q.category=category
        q.write_query(query_type,query_tree)
        q.print_tree(q.query)
        if not q.query or not all([sq.query for sq in q.subqueries]): #Smarts did not compile, sqs needed bc of hidden sqs in or queries
            bad_cats.add(category)
            if q.type=='b:StructureQuery':
                bad_smarts.add(q.smart)
        query_tree[query_id]=q
    all_tests[category]=query_tree[query_id]
    print_tests[category] = [query_id, query_type]#Final one should always be the top level query hopefully

('24', 'b:StructureQuery', '[#6,#16](=[#8])(Cl)[#6X4,c]', 'does not process')
('26', 'b:ParameterQuery', 'log Kow', 8.0, 'LessThan', 'does not process')
('29', 'b:ParameterQuery', 'Molecular weight', 1000.0, 'LessThan', 'does not process')
('32', 'LogicalQuery', 'And', 'does not process')
	('29', 'b:ParameterQuery', 'Molecular weight', 1000.0, 'LessThan', 'does not process')
	('26', 'b:ParameterQuery', 'log Kow', 8.0, 'LessThan', 'does not process')
	('24', 'b:StructureQuery', '[#6,#16](=[#8])(Cl)[#6X4,c]', 'does not process')
('43', 'b:StructureQuery', '[#6](=[#8])([#7h])[#6](=[#6h2])[Ch3,#1]', 'does not process')
('45', 'b:ParameterQuery', 'log Kow', 8.0, 'LessThan', 'does not process')
('48', 'b:ParameterQuery', 'Molecular weight', 1000.0, 'LessThan', 'does not process')
('51', 'LogicalQuery', 'And', 'does not process')
	('48', 'b:ParameterQuery', 'Molecular weight', 1000.0, 'LessThan', 'does not process')
	('45', 'b:ParameterQuery', 'log Kow', 8.0, 'LessThan', 'does not process')
	

RDKit ERROR: [13:31:50] SMARTS Parse Error: syntax error while parsing: [#7](=[#8])$[[#1],[#6X4]]{1..;xm}
RDKit ERROR: [13:31:50] SMARTS Parse Error: Failed parsing SMARTS '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}' for input: '[#7](=[#8])$[[#1],[#6X4]]{1..;xm}'
RDKit ERROR: [13:31:50] SMARTS Parse Error: syntax error while parsing: $[[#1],[#6X4]]{1..;xm}[#7+][#8-]
RDKit ERROR: [13:31:50] SMARTS Parse Error: Failed parsing SMARTS '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]' for input: '$[[#1],[#6X4]]{1..;xm}[#7+][#8-]'
RDKit ERROR: [13:31:50] SMARTS Parse Error: syntax error while parsing: c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}
RDKit ERROR: [13:31:50] SMARTS Parse Error: Failed parsing SMARTS 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}' for input: 'c1ccccc1.$[[#1],$[[#6X4]]{1..}]{1..;x}'
RDKit ERROR: [13:31:50] SMARTS Parse Error: syntax error while parsing: $[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]
RDKit ERROR: [13:31:50] SMARTS Parse Error: Failed parsing SMARTS '$[$[[#6X4]]{1..}]{1..;x}[Cl,Br,#8h]' for input: 

In [16]:
#print_tests

In [17]:
all_tests['Acid Chlorides']

In [18]:
tests=all_tests.copy()
for category in bad_cats:
    del tests[category]

In [19]:
tests['Acid Chlorides']

<h1>Fix bad SMARTS</h1>

In [20]:
bad_cats

{'Aliphatic Amines',
 'Alkoxysilanes',
 'Aminobenzothiazole Azo Dyes',
 'Anionic Surfactants',
 'Dianilines',
 'Dithiocarbamates (Acute toxicity)',
 'Dithiocarbamates (Chronic toxicity)',
 'Ethylene Glycol Ethers',
 'Neutral Organics',
 'Nonionic Surfactants',
 'Organotins (Acute toxicity)',
 'Organotins (Chronic toxicity)',
 'Persistent, Bioaccumulative and Toxic (PBT) Chemicals',
 'Polynitroaromatics (Acute toxicity)',
 'Polynitroaromatics (Chronic toxicity)',
 'Substituted Triazines (Acute toxicity)',
 'Substituted Triazines (Chronic toxicity)',
 'Triarylmethane Pigments/Dyes with Non-solubilizing Groups',
 'beta-Naphthylamines, Sulfonated'}

In [21]:
new_tests={}

In [22]:
#Aliphatic amines
def create_test():
    primamine=Chem.MolFromSmarts('[NX3;H2;!$(NC=[O,N,S]);!$(NCN)][CX3]')
    secamine=Chem.MolFromSmarts('[NX3;H1;!$(NC=[O,N,S]);!$(NCN)](C)[CX3]')
    tertamine=Chem.MolFromSmarts('[N;!$(NC=[O,N,S]);!$(NCN)](C)(C)[CX3]')
    def test(x):
        mol=x['mol']
        smiles=x['smiles']
        mw=x['mol_weight']
        return 'c' not in smiles and mw<1000 and '1' not in smiles and (mol.HasSubstructMatch(primamine) or mol.HasSubstructMatch(secamine)\
        or mol.HasSubstructMatch(tertamine)) 
    return test
new_tests['Aliphatic Amines']=create_test()

In [23]:
#Alkoxysilanes
def create_test():
    alkoxy=Chem.MolFromSmarts('[CX4]O[SiX4]')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        return mw<1000 and mol.HasSubstructMatch(alkoxy)
    return test
new_tests['Alkoxysilanes']=create_test()

In [24]:
#Aminobenzothiazole Azo Dyes
def create_test():
    azodye=Chem.MolFromSmiles('N=NC1=NC2=C(S1)C=CC=C2')
    def test(x):
        mol=x['mol'] 
        return mol.HasSubstructMatch(azodye)
    return test
new_tests['Aminobenzothiazole Azo Dyes']=create_test()

In [25]:
#Anionic Surfactants
def create_test():
    sulfate=Chem.MolFromSmarts('COS(=O)(=O)[OH,O-]')
    sulfonate=Chem.MolFromSmarts('CS(=O)(=O)[OH,O-]')
    phosphate=Chem.MolFromSmarts('COP([OH1])([OH1])=O')
    carboxylic=Chem.MolFromSmarts('[CX3;!$(Cc)](=O)[OX2H1]')
    silicic=Chem.MolFromSmarts('[Si][OX2H]')
    def test(x):
        mol=x['mol']
        smiles=x['smiles']
        if set(smiles)-set(['C','c','O','P','S','i','[',']','(',')','=']):
            return False
        m=re.compile('\(.?C.?\)')
        if m.findall(smiles):
            return False
        rgroup_indexes=[i for i,atom in enumerate(smiles) if atom=='C']
        return (mol.HasSubstructMatch(sulfate) or mol.HasSubstructMatch(sulfonate)\
        or mol.HasSubstructMatch(phosphate) or mol.HasSubstructMatch(carboxylic)\
        or mol.HasSubstructMatch(silicic))\
        and sorted(rgroup_indexes)==range(min(rgroup_indexes),max(rgroup_indexes)+1) #Tests for straight alkyl chains
    return test
new_tests['Anionic Surfactants']=create_test()

In [26]:
smiles='CCCCC(CC)CCC(CC(C)C)OS(O)(=O)=O'
rgroup_indexes=[i for i,atom in enumerate(smiles) if atom=='C']
sorted(rgroup_indexes)==range(min(rgroup_indexes),max(rgroup_indexes)+1)

False

In [27]:
#Benzotriazoles
def create_test():
    benzotriazole=Chem.MolFromSmarts('n1c2ccccc2nn1')
    def test(x):
        mol=x['mol']
        return mol.HasSubstructMatch(benzotriazole)
    return test
new_tests['Benzotriazoles']=create_test()

In [28]:
#Dianilines
def create_test():
    dianiline=Chem.MolFromSmarts('c1cc([NH2])ccc1[CH2,O,N,S]c1ccccc1')
    not_dianiline1=Chem.MolFromSmarts('c1ccccc1[A]~[A]')
    not_dianiline2=Chem.MolFromSmarts('c1ccccc1[A](c)c')
    def test(x):
        mol=x['mol']
        return not mol.HasSubstructMatch(not_dianiline1) and not mol.HasSubstructMatch(not_dianiline2)\
        and len(mol.GetSubstructMatches(dianiline))==2 #lol
    return test
new_tests['Dianilines']=create_test()

In [29]:
def create_test():
    ethylenebisdithiocarbamate=Chem.MolFromSmiles('SC(=S)NCCNC(=S)S')
    dithiocarbamates=[]
    for i in range(1,5):
        for j in range(1,5):
            mol=Chem.MolFromSmiles('C'*i + 'NC(=S)S' + 'C'*j)
            dithiocarbamates.append(mol)
    dithiocarbamates.append(ethylenebisdithiocarbamate)
    def test(x):
        mol=x['mol']
        return x['mol_weight']<1000 and x['logp']<5 and any([mol.HasSubstructMatch(dithiocarbamate) for dithiocarbamate in dithiocarbamates])
    return test
new_tests['Dithiocarbamates (Acute toxicity)']=create_test()
def create_test():
    ethylenebisdithiocarbamate=Chem.MolFromSmiles('SC(=S)NCCNC(=S)S')
    dithiocarbamates=[]
    for i in range(1,5):
        for j in range(1,5):
            mol=Chem.MolFromSmiles('C'*i + 'NC(=S)S' + 'C'*j)
            dithiocarbamates.append(mol)
    dithiocarbamates.append(ethylenebisdithiocarbamate)
    def test(x):
        mol=x['mol']
        return x['mol_weight']<1000 and x['logp']>=5 and x['logp']<19 and any([mol.HasSubstructMatch(dithiocarbamate) for dithiocarbamate in dithiocarbamates])
    return test
new_tests['Dithiocarbamates (Chronic toxicity)']=create_test()

In [30]:
#Ethylene Glycol Ethers
#Have to enumerate       
def create_test():
    match_mols=[]
    for i in range(1,8):
        for j in range(0,8):
            for k in range(1,4):
                smart='C'*i+'OCC'*k+'O'+'C'*j
                match_mols.append(Chem.MolFromSmiles(smart))
    phenyl_mols=[]
    for i in range(0,7):
        for k in range(1,3):
            for l in range(0,3): #Technically could be any number but this is difficult to implement
                phenyl_smart='c1ccccc1'+'C'*l+'OCC'*k+'O'+'C'*i
                phenyl_mols.append(Chem.MolFromSmiles(phenyl_smart))
    
    def test(x):
        smiles=x['smiles']
        if set(smiles)-set(['C','c','O','1','(',')']):
            return False
        if smiles.count('O')<2:
            return False
        os=[i for i,o in enumerate(smiles) if o=='O']
        between_os=[smiles[(start+1):end] for start,end in zip(os,os[1:])]
        if any([between!='CC' for between in between_os]):
            return False
        m=re.compile('1.*O.*1')
        if m.findall(smiles):
            return False
        carbon1=smiles[0:min(os)]
        carbon2=smiles[(max(os)+1):]
        if carbon1.count('C')>7 or carbon2.count('C')>7:
            return False
        if carbon1.count('c')>6 or carbon2.count('c')>6:
            return False
        if not carbon1 and not carbon2:
            return False
        else:
            return True
    return test
new_tests['Ethylene Glycol Ethers']=create_test()

In [31]:
#Neutral Organics
#Verhaar scheme, see paper called Classifying Environmental Pollutants
def create_test():
    def test(x):
        mol=x['mol']
        if mol.HasSubstructMatch(Chem.MolFromSmarts('[!C;!c;!N;!O;!F;!Cl;!Br,I]')): #Rule 0.1 and 1.1
            return False
        logp=x['logp']
        if logp>8:
            return False
        mw=x['mol_weight']
        if mw>1000:
            return False
        if not mol.HasSubstructMatch(Chem.MolFromSmarts('[!C;!c]')): #Rule 1.3
            return True
        elif not mol.HasSubstructMatch(Chem.MolFromSmarts('[!C;!c;!Cl;!Br;!F]'))\
        and not mol.HasSubstructMatch(Chem.MolFromSmarts('[Cl,Br,F]C[$(C=C),$(Cc)]')): #Rule 1.4
            return True
        elif not mol.HasSubstructMatch(Chem.MolFromSmarts('[!C;!c;!O;!Cl;!Br;!F]')): #Rule 1.5
            if mol.HasSubstructMatch(Chem.MolFromSmarts('COC'))\
            and not mol.HasSubstructMatch(Chem.MolFromSmarts('COOC'))\
            and not mol.HasSubstructMatch(Chem.MolFromSmarts('C1OC1')): #Rule 1.5.1 and 1.7
                return True
            elif mol.HasSubstructMatch(Chem.MolFromSmarts('[C;!$(C=O)][OH]'))\
            and not mol.HasSubstructMatch(Chem.MolFromSmarts('C=CCO'))\
            and not mol.HasSubstructMatch(Chem.MolFromSmarts('C#CCO'))\
            and not mol.HasSubstructMatch(Chem.MolFromSmarts('cCO')): #Rule 1.5.2, 1.5.3, and 1.7CCCCOCCOCCO
                return True
            elif mol.HasSubstructMatch(Chem.MolFromSmarts('[C;!$(CO)]=O'))\
            and not mol.HasSubstructMatch(Chem.MolFromSmarts('[$(cC),$(C=C)]C=O'))\
            and not mol.HasSubstructMatch(Chem.MolFromSmarts('[Cl,Br]C=O'))\
            and not mol.HasSubstructMatch(Chem.MolFromSmarts('[Cl,Br]CC=O')): #Rule 1.5.4 and 1.7
                return True
            else:
                return False
        elif not mol.HasSubstructMatch(Chem.MolFromSmarts('[!C;!N]'))\
        and mol.HasSubstructMatch(Chem.MolFromSmarts('C[NH,NH0]')): #Rule 1.6
            return True
        else: 
            return False
    return test
new_tests['Neutral Organics']=create_test()

In [32]:
#Nonionic Surfactants
# nonsurf1=Chem.MolFromSmarts('COCCO')
# nonsurf2=Chem.MolFromSmarts('COCCOC')
# def test(x):
#     mol=x['mol']
#     return mol.HasSubstructMatch(nonsurf1) or mol.HasSubstructMatch(nonsurf2)
import re
def test(x):
    smiles=x['smiles']
    if '(' in smiles:
        return False
    split_smiles=smiles.split('O')
    if len(split_smiles)==1:
        return False
    mol=x['mol']
    if not mol.HasSubstructMatch(Chem.MolFromSmiles('COC')) or mol.HasSubstructMatch(Chem.MolFromSmiles('C=O')):
        return False
    return not any([re.search(r'[^C]',c) for c in split_smiles])
new_tests['Nonionic Surfactants']=test

In [33]:
#Nonionic Surfactants
import math
def create_test():
    def test(x):
        mol=x['mol']
        atoms=[a for a in x['smiles'].lower() if a.isalpha()]
        if set(atoms)-set(['c','o']):
            return False
        return mol.HasSubstructMatch(Chem.MolFromSmarts('[CH3][CR0][CR0][CR0][CR0][CR0]')) and\
        atoms.count('o')>1 and\
        (math.floor(len(mol.GetSubstructMatches(Chem.MolFromSmarts('O[CH2][CH2]')))/2)+1)==len(mol.GetSubstructMatches(Chem.MolFromSmarts('[O]')))
    return test
new_tests['Nonionic Surfactants']=create_test()

In [34]:
#Organotins (Acute toxicity) and Organotins (Chronic toxicity)

def create_test():
    organotin=Chem.MolFromSmarts('C[Sn]') 
    def test(x):
        mol=x['mol']
        return x['mol_weight']<1000 and mol.HasSubstructMatch(organotin) and x['logp']<=13.7
    return test
new_tests['Organotins (Acute toxicity)']=create_test()
def create_test():
    organotin=Chem.MolFromSmarts('C[Sn]') 
    def test(x):
        mol=x['mol']
        return x['mol_weight']<1000 and mol.HasSubstructMatch(organotin) and x['logp']>=13.7
    return test
new_tests['Organotins (Chronic toxicity)']=create_test()

In [35]:
#Persistent, Bioaccumulative and Toxic (PBT) Chemicals
#MW<1000
#OPERA_HL > np.log(60)
#Ready biodegradability ?????
#LogP>4.2
#Not sure this one is worth doing without ready biodegradability
# def test(x):
#     mol=x['mol']
# new_tests['Persistent, Bioaccumulative and Toxic (PBT) Chemicals']=test

In [36]:
#Polynitroaromatics (Acute toxicity) and Polynitroaromatics (Chronic toxicity)
#MW < 1000

def create_test():
    polynitroaromatic=Chem.MolFromSmarts('ON(=O)[$(c1c(N(O)=O)cccc1),$(c1cc(N(O)=O)ccc1),$(c1ccc(N(O)=O)cc1),$(c1cncc(N(O)=O)c1)]')
    def test(x):
        mol=x['mol']
        return x['mol_weight']<1000 and mol.HasSubstructMatch(polynitroaromatic) and x['logp']<7
    return test
new_tests['Polynitroaromatics (Acute toxicity)']=create_test()
def create_test():
    polynitroaromatic=Chem.MolFromSmarts('N[$(c1c(N)cccc1),$(c1cc(N)ccc1),$(c1ccc(N)cc1),$(c1cncc(N)c1)]')
    def test(x):
        mol=x['mol']
        return x['mol_weight']<1000 and mol.HasSubstructMatch(polynitroaromatic) and x['logp']>=10
    return test
new_tests['Polynitroaromatics (Chronic toxicity)']=create_test()

In [37]:
#Substituted Triazines (Acute toxicity) and Substituted Triazines (Chronic toxicity)
#logp<5
#MW<1000
def create_test():
    subtriazine=Chem.MolFromSmarts('[$(n1nnccc1.[!#1]),$(n1ncncc1.[!#1]),$(n1cncnc1.[!#1])]')#[!H] did not work as expected with aromatics
    def test(x):
        mol=x['mol']
        return x['mol_weight']<1000 and mol.HasSubstructMatch(subtriazine) and x['logp']<5
    return test
new_tests['Substituted Triazines (Acute toxicity)']=create_test()
def create_test():
    subtriazine=Chem.MolFromSmarts('[$(n1nnccc1.[!#1]),$(n1ncncc1.[!#1]),$(n1cncnc1.[!#1])]')#[!H] did not work as expected with aromatics
    def test(x):
        mol=x['mol']
        return x['mol_weight']<1000 and mol.HasSubstructMatch(subtriazine) and x['logp']>5 and x['logp']<=8
    return test
new_tests['Substituted Triazines (Chronic toxicity)']=create_test()

In [38]:
def convert_ppb(x): #OPERA results stored as mol/L
    ws=x['ws']
    mol_weight=x['mol_weight']
    return ws*mol_weight*10**6

In [39]:
#Triarylmethane Pigments/Dyes with Non-solubilizing Groups
def create_test():
    para_permutations='[NH2,O,$([NH1][CH3]),$([NH1][CH2][CH3]),$(N([CH3])[CH3]),$(N([CH3])[CH2][CH3]),$(N([CH2][CH3])[CH2][CH3])]'
    triphenylmethane=Chem.MolFromSmarts('[cH]1[cH]c({})[cH][cH]c1C(c2[cH][cH]c({})[cH][cH]2)=C3[CH]=[CH]C(=[NH,O])[CH]=[CH]3'.format(para_permutations,para_permutations))
    diphenylnaphthylmethane=Chem.MolFromSmarts('[cH]1[cH]c({})[cH][cH]c1C(c2[cH][cH]c({})[cH]3[cH][cH][cH][cH][cH]32)=C3[CH]=[CH]C(=[NH,O])[CH]=[CH]3'.format(para_permutations,para_permutations))
    def test(x):
        mol=x['mol']
        return convert_ppb(x)>1 and (mol.HasSubstructMatch(triphenylmethane) or (mol.HasSubstructMatch(diphenylnaphthylmethane)))
    return test
new_tests['Triarylmethane Pigments/Dyes with Non-solubilizing Groups']=create_test()

In [40]:
#beta-Naphthylamines, Sulfonated
def create_test():
    smarts=[]
    match_mols=[]
    prefix='[NH2]c1[cH1,$(cO)]'
    suffix='[cH][cH]1'
    for c1 in range(1,4):
        for c2 in range(c1+1,5):
            smarts.append(prefix+'c2'+'[cH]'*(c1-1)+'[cH,$(c[OH]),$(c[NH2])]'+'[cH]'*(c2-c1-1)+'c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])'+'[cH]'*(4-c2)+'c2'+suffix)
            smarts.append(prefix+'c2'+'[cH]'*(c1-1)+'c([$(S(=O)(=O)[OH]),$(S(=O)(=O)[CH2][CH2]S[OH3])])'+'[cH]'*(c2-c1-1)+'[cH1,$(c[OH]),$(c[NH2])]'+'[cH]'*(4-c2)+'c2'+suffix)
    match_mols=[Chem.MolFromSmarts(smart) for smart in smarts]
    def test(x):
        mol=x['mol']
        naph_matches=[True for match in match_mols[:] if mol.HasSubstructMatch(match) and match.HasSubstructMatch(mol)]
        return any(naph_matches)
    return test
new_tests['beta-Naphthylamines, Sulfonated']=create_test()

In [41]:
#Aldehydes
#Turns out these are just wrong in the toolbox, although does compile
def create_test():
    formaldehyde=Chem.MolFromSmarts('[CH2](=O)') #Needs to be special case because buggy way RDKit handles hydrogens
    aldehyde=Chem.MolFromSmarts('[CH1](=[O])[C,c]')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        logp=x['logp']
        return (mol.HasSubstructMatch(formaldehyde) or mol.HasSubstructMatch(aldehyde)) and mw<1000 and logp<=6
    return test
new_tests['Aldehydes (Acute toxicity)']=create_test()
def create_test():
    formaldehyde=Chem.MolFromSmarts('[CH2](=O)') #Needs to be special case because buggy way RDKit handles hydrogens
    aldehyde=Chem.MolFromSmarts('[CH1](=[O])[C,c]')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        logp=x['logp']
        return (mol.HasSubstructMatch(formaldehyde) or mol.HasSubstructMatch(aldehyde)) and mw<1000 and logp>6
    return test
new_tests['Aldehydes (Chronic toxicity)']=create_test()

In [42]:
#Benzotriazoles
#Not a valid smarts from toolbox
def create_test():
    benzotriazole=Chem.MolFromSmarts('N1N=NC2=C1C=CC=C2')
    def test(x):
        mol=x['mol']
        return mol.HasSubstructMatch(benzotriazole)
    return test
new_tests['Benzotriazoles']=create_test()

In [43]:
#Imides
#Doesn't work if carbons are part of aromatic
def create_test():
    imide=Chem.MolFromSmarts('C(=O)NC(=O)')
    not_imide=Chem.MolFromSmarts('c1C(=O)NC(=O)ccccc1')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        logp=x['logp']
        return mol.HasSubstructMatch(imide) and not mol.HasSubstructMatch(not_imide) and logp<=5 and mw<1000
    return test
new_tests['Imides (Acute toxicity)']=create_test()
def create_test():
    imide=Chem.MolFromSmarts('C(=O)NC(=O)')
    not_imide=Chem.MolFromSmarts('c1(C(=O)NC(=O))ccccc1')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        logp=x['logp']
        return mol.HasSubstructMatch(imide) and not mol.HasSubstructMatch(not_imide) and logp>5 and logp<8 and mw<1000
    return test
new_tests['Imides (Chronic toxicity)']=create_test()

In [44]:
#Hydrazines and related compounds
def create_test():
    hydra1=Chem.MolFromSmarts('[NX3][NX3]')
    hydra2=Chem.MolFromSmarts('[CX3]=[NX2][NX2]')
    hydra3=Chem.MolFromSmarts('[CX3](=O)[NX2][NX3]')
    hydra4=Chem.MolFromSmarts('[NX2][CX3](=O)[NX2][NX3]')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        return (mol.HasSubstructMatch(hydra1) or mol.HasSubstructMatch(hydra2)\
               or mol.HasSubstructMatch(hydra3) or mol.HasSubstructMatch(hydra4)) and mw<500
    return test
new_tests['Hydrazines and Related Compounds']=create_test()

In [45]:
#Thiols
def create_test():
    thiol=Chem.MolFromSmarts('[C,c][SX2H]')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        logp=x['logp']
        return mol.HasSubstructMatch(thiol) and mw<1000 and logp<6.5
    return test
new_tests['Thiols (Acute toxicity)']=create_test()
def create_test():
    thiol=Chem.MolFromSmarts('[C,c][SX2H]')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        logp=x['logp']
        return mol.HasSubstructMatch(thiol) and mw<1000 and logp>=6.5 and logp<9
    return test
new_tests['Thiols (Chronic toxicity)']=create_test()

In [46]:
#Acrylamides
def create_test():
    acrylamide1=Chem.MolFromSmarts('[CH2]=[CH1]C(=O)[NH,NH2]')
    acrylamide2=Chem.MolFromSmarts('[CH2]=C([CH3])C(=O)[NH,NH2]')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        logp=x['logp']
        return (mol.HasSubstructMatch(acrylamide1) or mol.HasSubstructMatch(acrylamide2)) and mw<1000 and logp<8
    return test
new_tests['Acrylamides']=create_test()

In [47]:
#Acrylates/Methacrylates
def create_test():
    acrylate=Chem.MolFromSmarts('[CH2]=[CH]C(=O)O')
    methacrylate=Chem.MolFromSmarts('[CH2]=C([CH3])C(=O)O')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        logp=x['logp']
        return (mol.HasSubstructMatch(acrylate) or mol.HasSubstructMatch(methacrylate)) and logp<=5 and mw<1000
    return test
new_tests['Acrylates/Methacrylates (Acute toxicity)']=create_test()
def create_test():
    acrylate=Chem.MolFromSmarts('[CH2]=[CH]C(=O)O')
    methacrylate=Chem.MolFromSmarts('[CH2]=C([CH3])C(=O)O')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        logp=x['logp']
        return (mol.HasSubstructMatch(acrylate) or mol.HasSubstructMatch(methacrylate)) and logp>5 and logp<8 and mw<1000
    return test
new_tests['Acrylates/Methacrylates (Chronic toxicity)']=create_test()

In [48]:
#Epoxides
#Grace's advice
def create_test():
    epoxide=Chem.MolFromSmarts('c1oc1')
    aziridine=Chem.MolFromSmarts('c1cn1([CH3,$(CH2CH3)])')
    def test(x):
        mol=x['mol']
        mw=x['mol_weight']
        return mw<1000 and (mol.HasSubstructMatch(epoxide) or mol.HasSubstructMatch(aziridine))
    return test
new_tests['Epoxides']=create_test()

RDKit ERROR: [13:32:35] SMARTS Parse Error: syntax error while parsing: c1cn1([CH3,$(CH2CH3)_100])
RDKit ERROR: [13:32:35] SMARTS Parse Error: Failed parsing SMARTS 'c1cn1([CH3,$(CH2CH3)_100])' for input: 'c1cn1([CH3,$(CH2CH3)])'


In [49]:
new_tests.update({k:q.query for k,q in tests.items()})

In [50]:
len(new_tests)

66

In [51]:
sorted(list(new_tests.keys()))[:10]

['Acid Chlorides',
 'Acrylamides',
 'Acrylates/Methacrylates (Acute toxicity)',
 'Acrylates/Methacrylates (Chronic toxicity)',
 'Aldehydes (Acute toxicity)',
 'Aldehydes (Chronic toxicity)',
 'Aliphatic Amines',
 'Alkoxysilanes',
 'Aluminum Compounds',
 'Aminobenzothiazole Azo Dyes']

In [52]:
import pymongo
from pymongo import MongoClient

In [55]:
client = MongoClient('mongodb://localhost:27017/')

In [56]:
client.list_database_names()

['admin',
 'config',
 'flights',
 'genra_dev_v5',
 'genra_dev_v5_1',
 'local',
 'toxval']

In [57]:
DB = client.genra_dev_v5

In [58]:
DB.list_collection_names()

['htpp_cr_fp_0',
 'toxval_v9',
 'htpp_cr_fp_1',
 'httr_cr_fp_0',
 'compounds',
 'chms_fp',
 'toxref_effects',
 'toxcast_assays',
 'htpp_fp_1',
 'toxref_tr_fp',
 'fp_stats',
 'toxcast_fp',
 'toxval_v9_1_1']

In [59]:
DB2 = client.genra_dev_v5_1
DB2.list_collection_names()

['htpp_cr_fp_0',
 'physprop',
 'toxval',
 'toxcast_assays',
 'toxcast_fp',
 'toxref_effects',
 'httr_cr_fp_0',
 'compounds',
 'chms_fp',
 'toxval_v9_1_1',
 'toxref_tr_fp',
 'chmt']

In [60]:
#[e for e in DB2.physprop.find({'dsstox_sid': 'DTXSID8059718'})]

In [61]:
loael_smiles=DB.compounds.find({},{'_id':0,'dsstox_sid':1,'smiles':1}).limit(10)

In [62]:
smiles_dict={record['dsstox_sid']:record['smiles'] for record in loael_smiles if record['smiles']}

In [63]:
smiles_dict

{'DTXSID7020001': 'NC1=NC2=C(C=C1)C1=CC=CC=C1N2',
 'DTXSID2020004': 'CC=NO',
 'DTXSID7020005': 'CC(N)=O',
 'DTXSID2020006': 'CC(=O)NC1=CC=C(O)C=C1',
 'DTXSID7020007': 'CC(=O)C1=CC=C(C=C1)S(=O)(=O)NC(=O)NC1CCCCC1',
 'DTXSID2020008': 'CC(C)=NNC1=NC=C(S1)C1=CC=C(O1)[N+]([O-])=O',
 'DTXSID7020009': 'CC#N',
 'DTXSID6020010': 'CC(C)=NO',
 'DTXSID6020012': 'CC(=O)NNC1=CC=C(CO)C=C1',
 'DTXSID1020013': 'CC(=O)NNC(=O)C1=CC=NC=C1'}

In [64]:
#[e for e in DB2.physprop.find({},{'_id':0,'dsstox_sid':1,'predicted_props.OPERA_LogP':1}).limit(10)]

In [65]:
DB2.physprop.find_one({})

{'_id': ObjectId('606728b382812d061e3c323b'),
 'dsstox_sid': 'DTXSID001000007',
 'dsstox_cid': 'DTXCID101426975',
 'predicted_props': {'OPERA_WS': [0.0282756],
  'OPERA_BP': [173.703],
  'OPERA_HL': [2.33995e-05],
  'OPERA_KOA': [3.89446],
  'OPERA_LogP': [1.50611],
  'OPERA_MP': [13.0102],
  'OPERA_PKAA': [6.56497],
  'OPERA_VP': [1.08764]}}

In [66]:
loael_logp = [e for e in DB2.physprop.find({'dsstox_sid': {'$in': list(smiles_dict.keys())}}, {"dsstox_sid":1, "predicted_props.OPERA_LogP" : 1, "_id":0})]

In [67]:
loael_logp[0]

{'dsstox_sid': 'DTXSID1020013', 'predicted_props': {'OPERA_LogP': [-0.284827]}}

In [68]:
logp_dict = {record['dsstox_sid']: record.get('predicted_props',{})['OPERA_LogP'][0] for record in loael_logp}

In [69]:
logp_dict

{'DTXSID1020013': -0.284827,
 'DTXSID2020004': -0.12933,
 'DTXSID2020006': 0.462113,
 'DTXSID2020008': 4.34955,
 'DTXSID6020010': 0.119381,
 'DTXSID6020012': 0.576014,
 'DTXSID7020005': -1.17478,
 'DTXSID7020001': 2.59894,
 'DTXSID7020007': 2.43949,
 'DTXSID7020009': -0.33971}

In [70]:
loael_ws = [e for e in DB2.physprop.find({'dsstox_sid': {'$in': list(smiles_dict.keys())}}, {"dsstox_sid":1, "predicted_props.OPERA_WS" : 1, "_id":0})]

In [71]:
ws_dict = {record['dsstox_sid']: record.get('predicted_props',{})['OPERA_WS'][0] for record in loael_ws}
ws_dict

{'DTXSID1020013': 0.205315,
 'DTXSID2020004': 0.183241,
 'DTXSID2020006': 0.0933539,
 'DTXSID2020008': 2.25893e-05,
 'DTXSID6020010': 0.21844,
 'DTXSID6020012': 0.054174,
 'DTXSID7020005': 37.6989,
 'DTXSID7020001': 0.00475859,
 'DTXSID7020007': 0.00862582,
 'DTXSID7020009': 12.6417}

In [72]:
DB.compounds.find_one({})

{'_id': ObjectId('609430c44ce714020b32837e'),
 'casrn': '26148-68-5',
 'chemspider_id': 56541.0,
 'dsstox_cid': 'DTXCID101',
 'dsstox_sid': 'DTXSID7020001',
 'gsid': 20001,
 'inchi_key': 'FJTNLJLPLJDTRM-UHFFFAOYSA-N',
 'iupac': '9H-Pyrido[2,3-b]indol-2-amine',
 'mol_weight': 183.214,
 'name': 'A-alpha-C',
 'pubchem_cid': 62805.0,
 'smiles': 'NC1=NC2=C(C=C1)C1=CC=CC=C1N2'}

In [73]:
loael_weight = [e for e in DB.compounds.find({'dsstox_sid': {'$in': list(smiles_dict.keys())}}, {"dsstox_sid":1, "mol_weight" : 1, "_id":0})]

In [74]:
loael_weight

[{'dsstox_sid': 'DTXSID1020013', 'mol_weight': 179.179},
 {'dsstox_sid': 'DTXSID2020004', 'mol_weight': 59.068},
 {'dsstox_sid': 'DTXSID2020006', 'mol_weight': 151.165},
 {'dsstox_sid': 'DTXSID2020008', 'mol_weight': 266.28},
 {'dsstox_sid': 'DTXSID6020010', 'mol_weight': 73.095},
 {'dsstox_sid': 'DTXSID6020012', 'mol_weight': 180.207},
 {'dsstox_sid': 'DTXSID7020001', 'mol_weight': 183.214},
 {'dsstox_sid': 'DTXSID7020005', 'mol_weight': 59.068},
 {'dsstox_sid': 'DTXSID7020007', 'mol_weight': 324.4},
 {'dsstox_sid': 'DTXSID7020009', 'mol_weight': 41.053}]

In [75]:

weight_dict={record['dsstox_sid']:record['mol_weight'] for record in loael_weight}

In [76]:
weight_dict

{'DTXSID1020013': 179.179,
 'DTXSID2020004': 59.068,
 'DTXSID2020006': 151.165,
 'DTXSID2020008': 266.28,
 'DTXSID6020010': 73.095,
 'DTXSID6020012': 180.207,
 'DTXSID7020001': 183.214,
 'DTXSID7020005': 59.068,
 'DTXSID7020007': 324.4,
 'DTXSID7020009': 41.053}

In [77]:
sids=set(logp_dict.keys())&set(ws_dict.keys())&set(weight_dict.keys())&set(smiles_dict.keys())

In [79]:
sids

{'DTXSID1020013',
 'DTXSID2020004',
 'DTXSID2020006',
 'DTXSID2020008',
 'DTXSID6020010',
 'DTXSID6020012',
 'DTXSID7020001',
 'DTXSID7020005',
 'DTXSID7020007',
 'DTXSID7020009'}

In [80]:

records=[]
for sid in sids:
    records.append({'dsstox_sid':sid,'smiles':smiles_dict[sid],'logp':logp_dict[sid],'ws':ws_dict[sid],'mol_weight':weight_dict[sid],'mol':Chem.MolFromSmiles(smiles_dict[sid])})
records=[record for record in records if record['mol']]

In [81]:
records[0]

{'dsstox_sid': 'DTXSID7020009',
 'smiles': 'CC#N',
 'logp': -0.33971,
 'ws': 12.6417,
 'mol_weight': 41.053,
 'mol': <rdkit.Chem.rdchem.Mol at 0x7f633583fc60>}

In [82]:
import math
for record in records:
    #print(record)
    epa_categories=sorted([category for category,test in new_tests.items() if test(record)])
    if 'Neutral Organics' in epa_categories and len(epa_categories)>1:
        epa_categories.remove('Neutral Organics')
    record['categories']=tuple(epa_categories)
    

In [83]:
pd.DataFrame(records)

,dsstox_sid,smiles,logp,ws,mol_weight,mol,categories
0,DTXSID7020009,CC#N,-0.339710,12.641700,41.053,<rdkit.Chem.rdchem.Mol object at 0x7f633583fc60>,()
1,DTXSID6020010,CC(C)=NO,0.119381,0.218440,73.095,<rdkit.Chem.rdchem.Mol object at 0x7f6335843b70>,()
2,DTXSID7020005,CC(N)=O,-1.174780,37.698900,59.068,<rdkit.Chem.rdchem.Mol object at 0x7f6335843c10>,()
3,DTXSID1020013,CC(=O)NNC(=O)C1=CC=NC=C1,-0.284827,0.205315,179.179,<rdkit.Chem.rdchem.Mol object at 0x7f6335843c60>,"(Hydrazines and Related Compounds,)"
4,DTXSID7020007,CC(=O)C1=CC=C(C=C1)S(=O)(=O)NC(=O)NC1CCCCC1,2.439490,0.008626,324.400,<rdkit.Chem.rdchem.Mol object at 0x7f6335843cb0>,()
5,DTXSID2020006,CC(=O)NC1=CC=C(O)C=C1,0.462113,0.093354,151.165,<rdkit.Chem.rdchem.Mol object at 0x7f6335843d00>,"(Phenols (Acute toxicity),)"
6,DTXSID2020004,CC=NO,-0.129330,0.183241,59.068,<rdkit.Chem.rdchem.Mol object at 0x7f6335843d50>,()
7,DTXSID7020001,NC1=NC2=C(C=C1)C1=CC=CC=C1N2,2.598940,0.004759,183.214,<rdkit.Chem.rdchem.Mol object at 0x7f6335843da0>,()
8,DTXSID2020008,CC(C)=NNC1=NC=C(S1)C1=CC=C(O1)[N+]([O-])=O,4.349550,0.000023,266.280,<rdkit.Chem.rdchem.Mol object at 0x7f6335843df0>,"(Hydrazines and Related Compounds,)"
9,DTXSID6020012,CC(=O)NNC1=CC=C(CO)C=C1,0.576014,0.054174,180.207,<rdkit.Chem.rdchem.Mol object at 0x7f6335843e40>,"(Hydrazines and Related Compounds,)"


In [140]:
tsca_df = pd.read_excel(interim_dir+'TSCA_OPERA_predictions_080722.xlsx')

In [87]:
records[0].keys()

dict_keys(['dsstox_sid', 'smiles', 'logp', 'ws', 'mol_weight', 'mol', 'categories'])

In [141]:
tsca_df2 = tsca_df[tsca_df['errors'].isnull()]

In [95]:
tsca_df2 = tsca_df2.set_index('dtxsid')

In [112]:
tsca_df2.columns

Index(['Unnamed: 0', 'PREFERRED_NAME', 'CASRN', 'INCHIKEY', 'IUPAC_NAME',
       'SMILES', 'INCHI_STRING', 'MOLECULAR_FORMULA', 'AVERAGE_MASS',
       'MONOISOTOPIC_MASS', 'DATA_SOURCES', 'NUMBER_OF_PUBMED_ARTICLES',
       'PUBCHEM_DATA_SOURCES', 'CPDAT_COUNT', 'MolWeight', 'nbAtoms',
       'nbHeavyAtoms', 'nbC', 'nbO', 'nbN', 'nbAromAtom', 'nbRing',
       'nbHeteroRing', 'Sp3Sp2HybRatio', 'nbRotBd', 'nbHBdAcc', 'ndHBdDon',
       'nbLipinskiFailures', 'TopoPolSurfAir', 'MolarRefract',
       'CombDipolPolariz', 'LogP_pred', 'LogP_predRange', 'AD_LogP',
       'AD_index_LogP', 'Conf_index_LogP', 'MP_pred', 'MP_predRange', 'AD_MP',
       'AD_index_MP', 'Conf_index_MP', 'BP_pred', 'BP_predRange', 'AD_BP',
       'AD_index_BP', 'Conf_index_BP', 'LogVP_pred', 'VP_predRange', 'AD_VP',
       'AD_index_VP', 'Conf_index_VP', 'LogWS_pred', 'WS_predRange', 'AD_WS',
       'AD_index_WS', 'Conf_index_WS', 'LogHL_pred', 'HL_predRange', 'AD_HL',
       'AD_index_HL', 'Conf_index_HL', 'RT_pred',

In [120]:
test_df = tsca_df2[['qsar_ready_smiles','LogP_pred','WS_pred_mol/L','MolWeight']].head()

In [121]:
test_df = test_df.reset_index()

In [122]:
test_df

,dtxsid,qsar_ready_smiles,LogP_pred,WS_pred_mol/L,MolWeight
0,DTXSID3060164,C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1,5.76,4.073803e-07,244.125201
1,DTXSID7060837,ICCCI,3.02,7.413102e-04,295.855896
2,DTXSID9025879,OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1,1.99,9.120108e-03,218.057909
3,DTXSID2026282,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,3.11,2.454709e-08,667.174336
4,DTXSID4052188,CC1(C)COC(C)(OC1)C1C=CC=CC=1,2.72,1.148154e-03,206.130680


In [123]:
test_df.columns = ['dsstox_sid', 'smiles', 'logp', 'ws', 'mol_weight']

In [126]:
test_df['mol'] = [Chem.MolFromSmiles(e) for e in test_df['smiles']]

In [127]:
test_df

,dsstox_sid,smiles,logp,ws,mol_weight,mol
0,DTXSID3060164,C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1,5.76,4.073803e-07,244.125201,<rdkit.Chem.rdchem.Mol object at 0x7f63272b3080>
1,DTXSID7060837,ICCCI,3.02,7.413102e-04,295.855896,<rdkit.Chem.rdchem.Mol object at 0x7f63272b38f0>
2,DTXSID9025879,OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1,1.99,9.120108e-03,218.057909,<rdkit.Chem.rdchem.Mol object at 0x7f63272b37b0>
3,DTXSID2026282,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,3.11,2.454709e-08,667.174336,<rdkit.Chem.rdchem.Mol object at 0x7f63272b3a80>
4,DTXSID4052188,CC1(C)COC(C)(OC1)C1C=CC=CC=1,2.72,1.148154e-03,206.130680,<rdkit.Chem.rdchem.Mol object at 0x7f63272b3f30>


In [130]:
test_dict = test_df.to_dict('records')

In [133]:
test_dict

[{'dsstox_sid': 'DTXSID3060164',
  'smiles': 'C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1',
  'logp': 5.76,
  'ws': 4.07380277804113e-07,
  'mol_weight': 244.125200512,
  'mol': <rdkit.Chem.rdchem.Mol at 0x7f63272b3080>},
 {'dsstox_sid': 'DTXSID7060837',
  'smiles': 'ICCCI',
  'logp': 3.02,
  'ws': 0.0007413102413009177,
  'mol_weight': 295.855896192,
  'mol': <rdkit.Chem.rdchem.Mol at 0x7f63272b38f0>},
 {'dsstox_sid': 'DTXSID9025879',
  'smiles': 'OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1',
  'logp': 1.99,
  'ws': 0.009120108393559097,
  'mol_weight': 218.0579088,
  'mol': <rdkit.Chem.rdchem.Mol at 0x7f63272b37b0>},
 {'dsstox_sid': 'DTXSID2026282',
  'smiles': 'O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=2C4=O)NC(=O)C2C=CC=CC=2)C2=C1C(=O)C1C=CC=CC=1C2=O)C1C=CC=CC=1',
  'logp': 3.11,
  'ws': 2.454708915685029e-08,
  'mol_weight': 667.174335520001,
  'mol': <rdkit.Chem.rdchem.Mol at 0x7f63272b3a80>},
 {'dsstox_sid': 'DTXSID4052188',
  'smiles': 'CC1(C)COC(C)(OC1)C1C=CC=CC=1',
  'logp': 2.72,
  'ws': 0

In [134]:
for record in test_dict:
    #print(record)
    epa_categories=sorted([category for category,test in new_tests.items() if test(record)])
    if 'Neutral Organics' in epa_categories and len(epa_categories)>1:
        epa_categories.remove('Neutral Organics')
    record['categories']=tuple(epa_categories)
    

In [136]:
pd.DataFrame(test_dict)

,dsstox_sid,smiles,logp,ws,mol_weight,mol,categories
0,DTXSID3060164,C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1,5.76,4.073803e-07,244.125201,<rdkit.Chem.rdchem.Mol object at 0x7f63272b3080>,"(Neutral Organics,)"
1,DTXSID7060837,ICCCI,3.02,7.413102e-04,295.855896,<rdkit.Chem.rdchem.Mol object at 0x7f63272b38f0>,()
2,DTXSID9025879,OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1,1.99,9.120108e-03,218.057909,<rdkit.Chem.rdchem.Mol object at 0x7f63272b37b0>,()
3,DTXSID2026282,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,3.11,2.454709e-08,667.174336,<rdkit.Chem.rdchem.Mol object at 0x7f63272b3a80>,()
4,DTXSID4052188,CC1(C)COC(C)(OC1)C1C=CC=CC=1,2.72,1.148154e-03,206.130680,<rdkit.Chem.rdchem.Mol object at 0x7f63272b3f30>,"(Neutral Organics,)"


In [142]:
epa_cats_df = tsca_df2[['dtxsid','qsar_ready_smiles','LogP_pred','WS_pred_mol/L','MolWeight']]

In [143]:
epa_cats_df

,dtxsid,qsar_ready_smiles,LogP_pred,WS_pred_mol/L,MolWeight
0,DTXSID3060164,C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1,5.76,4.073803e-07,244.125201
1,DTXSID7060837,ICCCI,3.02,7.413102e-04,295.855896
2,DTXSID9025879,OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1,1.99,9.120108e-03,218.057909
3,DTXSID2026282,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,3.11,2.454709e-08,667.174336
4,DTXSID4052188,CC1(C)COC(C)(OC1)C1C=CC=CC=1,2.72,1.148154e-03,206.130680
...,...,...,...,...,...
14242,DTXSID5064209,OC(=O)CSC1=NC2=CC=CC=C2S1,1.54,9.772372e-04,224.991820
14243,DTXSID7062873,COC(=O)CCCCl,1.55,2.041738e-01,136.029107
14244,DTXSID9040342,CCC1COC(=O)O1,0.03,9.772372e-01,116.047344
14245,DTXSID3044889,CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,4.26,3.981072e-03,220.146330


In [144]:
epa_cats_df.columns = ['dsstox_sid', 'smiles', 'logp', 'ws', 'mol_weight']

In [145]:
epa_cats_df['mol'] = [Chem.MolFromSmiles(e) for e in epa_cats_df['smiles']]

<ipython-input-145-402dac51a3bc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  epa_cats_df['mol'] = [Chem.MolFromSmiles(e) for e in epa_cats_df['smiles']]


In [149]:
epa_cats_df[epa_cats_df['mol'].isnull()]

,dsstox_sid,smiles,logp,ws,mol_weight,mol


In [150]:
epa_cats_dict = epa_cats_df.to_dict('records')

In [151]:
for record in epa_cats_dict:
    #print(record)
    epa_categories=sorted([category for category,test in new_tests.items() if test(record)])
    if 'Neutral Organics' in epa_categories and len(epa_categories)>1:
        epa_categories.remove('Neutral Organics')
    record['categories']=tuple(epa_categories)

In [153]:
epa_cats_dict_df = pd.DataFrame(epa_cats_dict)

In [160]:
epa_cats_dict_df.columns

Index(['dsstox_sid', 'smiles', 'logp', 'ws', 'mol_weight', 'mol',
       'categories'],
      dtype='object')

In [155]:
epa_cats_dict_df['categories'].value_counts()

()                                                                                                                      5699
(Neutral Organics,)                                                                                                     2667
(Esters (Acute toxicity),)                                                                                              1310
(Phenols (Acute toxicity),)                                                                                              502
(Anilines (Acute toxicity),)                                                                                             407
                                                                                                                        ... 
(Epoxides, Hindered Amines)                                                                                                1
(Alkoxysilanes, Anilines (Acute toxicity))                                                                                 1


In [162]:
epa_dict = {k:v for k,v in zip(epa_cats_dict_df['dsstox_sid'],epa_cats_dict_df['categories'] )}

In [172]:
epa_dict = {k:v for k,v in epa_dict.items() if len(v) != 0}

#### Add NCC tag to the main TSCA set

In [158]:
tsca_df = tsca_df.drop('Unnamed: 0', axis = 1)

In [182]:
tsca_df['NCC'] = tsca_df['dtxsid'].apply(lambda x : epa_dict[x] if x in epa_dict.keys() else np.nan) 
        

In [185]:
tsca_df['NCC'].value_counts(dropna = False)

NaN                                                                                      6647
(Neutral Organics,)                                                                      2667
(Esters (Acute toxicity),)                                                               1310
(Phenols (Acute toxicity),)                                                               502
(Anilines (Acute toxicity),)                                                              407
                                                                                         ... 
(Alkoxysilanes, Azides (Acute toxicity))                                                    1
(Aldehydes (Acute toxicity), Nonionic Surfactants)                                          1
(Phenols (Acute toxicity), Triarylmethane Pigments/Dyes with Non-solubilizing Groups)       1
(Esters (Acute toxicity), Substituted Triazines (Acute toxicity))                           1
(Aldehydes (Acute toxicity), Anilines (Acute toxicity))     

#### Tag DTXSID by vendor number to give an indicator of procurability

In [187]:
vendors = pd.read_excel(raw_dir+'TSCA_Active_ChemicalVendors_Summary_0218_2022.xlsx')

In [191]:
vendors.rename(columns = {'DTXSID': 'dtxsid'}, inplace = True)

In [192]:
vendors.columns

Index(['dtxsid', 'PREFERRED_NAME', 'CASRN', 'INCHIKEY', 'PUBCHEM_CID',
       'VENDOR_COUNT', 'IN_SIGMA-ALDRICH'],
      dtype='object')

In [194]:
final_input_df = pd.merge(tsca_df, vendors[['dtxsid','VENDOR_COUNT', 'IN_SIGMA-ALDRICH']],  on = 'dtxsid', how = 'left')

In [210]:
final_input_df

,dtxsid,PREFERRED_NAME,CASRN,INCHIKEY,IUPAC_NAME,SMILES,INCHI_STRING,MOLECULAR_FORMULA,AVERAGE_MASS,MONOISOTOPIC_MASS,...,errors,qsar_ready_smiles,HLC,WS_pred_mol/L,WS_pred_mg/L,test_track,physical_form,NCC,VENDOR_COUNT,IN_SIGMA-ALDRICH
0,DTXSID3060164,Benzhydrylbenzene,519-73-3,AAAQKTZKLRYKHR-UHFFFAOYSA-N,"1,1',1''-Methanetriyltribenzene",C1=CC=C(C=C1)C(C1=CC=CC=C1)C1=CC=CC=C1,InChI=1S/C19H16/c1-4-10-16(11-5-1)19(17-12-6-2...,C19H16,244.337,244.125201,...,NaN,C1=CC=CC=C1C(C1C=CC=CC=1)C1C=CC=CC=1,2.630268e-06,4.073803e-07,0.099452,B,solid,"(Neutral Organics,)",75.0,1.0
1,DTXSID7060837,"Propane, 1,3-diiodo-",627-31-6,AAAXMNYUNVCMCJ-UHFFFAOYSA-N,"1,3-Diiodopropane",ICCCI,InChI=1S/C3H6I2/c4-2-1-3-5/h1-3H2,C3H6I2,295.890,295.855890,...,NaN,ICCCI,2.454709e-03,7.413102e-04,219.321006,B,liquid,NaN,72.0,1.0
2,DTXSID9025879,"3,3'-(p-Phenylene)diacrylic acid",16323-43-6,AAFXQFIGKBLKMC-UHFFFAOYSA-N,"3,3'-(1,4-Phenylene)di(prop-2-enoic acid)",OC(=O)C=CC1=CC=C(C=CC(O)=O)C=C1,InChI=1S/C12H10O4/c13-11(14)7-5-9-1-2-10(4-3-9...,C12H10O4,218.208,218.057909,...,NaN,OC(=O)C=CC1C=CC(C=CC(O)=O)=CC=1,6.025596e-10,9.120108e-03,1988.711764,B,solid,NaN,13.0,0.0
3,DTXSID2026282,C.I. Vat brown 3,131-92-0,AAKMSGQPNUGLAZ-UHFFFAOYSA-N,"N,N'-(5,10,15,17-Tetraoxo-10,15,16,17-tetrahyd...",O=C(NC1=CC=CC2=C1C(=O)C1=C(C3=C(C=C1)C1=C(N3)C...,InChI=1S/C42H23N3O6/c46-37-24-14-7-8-15-25(24)...,C42H23N3O6,665.661,665.158685,...,NaN,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,2.630268e-09,2.454709e-08,0.016377,B,solid,NaN,27.0,0.0
4,DTXSID2026282,C.I. Vat brown 3,131-92-0,AAKMSGQPNUGLAZ-UHFFFAOYSA-N,"N,N'-(5,10,15,17-Tetraoxo-10,15,16,17-tetrahyd...",O=C(NC1=CC=CC2=C1C(=O)C1=C(C3=C(C=C1)C1=C(N3)C...,InChI=1S/C42H23N3O6/c46-37-24-14-7-8-15-25(24)...,C42H23N3O6,665.661,665.158685,...,NaN,O=C(NC1=CC2=C(NC3C4=C(C=CC2=3)C(=O)C2C(=CC=CC=...,2.630268e-09,2.454709e-08,0.016377,B,solid,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14972,DTXSID5064209,2-(Carboxymethylthio)benzothiazole,6295-57-4,ZZUQWNYNSKJLPI-UHFFFAOYSA-N,"[(1,3-Benzothiazol-2-yl)sulfanyl]acetic acid",OC(=O)CSC1=NC2=CC=CC=C2S1,InChI=1S/C9H7NO2S2/c11-8(12)5-13-9-10-6-3-1-2-...,C9H7NO2S2,225.280,224.991821,...,NaN,OC(=O)CSC1=NC2=CC=CC=C2S1,3.388442e-09,9.772372e-04,219.870381,B,solid,NaN,79.0,1.0
14973,DTXSID7062873,"Butanoic acid, 4-chloro-, methyl ester",3153-37-5,ZZUYIRISBMWFMV-UHFFFAOYSA-N,Methyl 4-chlorobutanoate,COC(=O)CCCCl,"InChI=1S/C5H9ClO2/c1-8-5(7)3-2-4-6/h2-4H2,1H3",C5H9ClO2,136.580,136.029107,...,NaN,COC(=O)CCCCl,2.818383e-04,2.041738e-01,27773.578977,B,liquid,"(Esters (Acute toxicity),)",71.0,1.0
14974,DTXSID9040342,"4-Ethyl-1,3-dioxolan-2-one",4437-85-8,ZZXUZKXVROWEIF-UHFFFAOYSA-N,"4-Ethyl-1,3-dioxolan-2-one",CCC1COC(=O)O1,"InChI=1/C5H8O3/c1-2-4-3-7-5(6)8-4/h4H,2-3H2,1H3",C5H8O3,116.116,116.047344,...,NaN,CCC1COC(=O)O1,2.570396e-08,9.772372e-01,113405.784063,B,liquid,"(Neutral Organics,)",49.0,1.0
14975,DTXSID3044889,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",2460-77-7,ZZYASVWWDLJXIM-UHFFFAOYSA-N,"2,5-Di-tert-butylcyclohexa-2,5-diene-1,4-dione",CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,"InChI=1S/C14H20O2/c1-13(2,3)9-7-12(16)10(8-11(...",C14H20O2,220.312,220.146330,...,NaN,CC(C)(C)C1=CC(=O)C(=CC1=O)C(C)(C)C,5.370318e-04,3.981072e-03,876.418325,B,solid,NaN,61.0,1.0


In [195]:
writer = pd.ExcelWriter(interim_dir+'tsca_struct_input_080722.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.

final_input_df.to_excel(writer)

writer.save()
